In [25]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import os
import sys

import api

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import County

## Database connection

In [27]:
engine, session = db.connect()
engine.echo = False

2023-03-24 16:18:40,837 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-24 16:18:40,838 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:18:40,839 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-24 16:18:40,839 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:18:40,840 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-24 16:18:40,840 INFO sqlalchemy.engine.Engine [raw sql] {}


## Create table County and download e inserted info of API

In [28]:
response = api.get_data_api()
response.json()

[{'id': 1100015,
  'nome': "Alta Floresta D'Oeste",
  'microrregiao': {'id': 11006,
   'nome': 'Cacoal',
   'mesorregiao': {'id': 1102,
    'nome': 'Leste Rondoniense',
    'UF': {'id': 11,
     'sigla': 'RO',
     'nome': 'Rondônia',
     'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}}},
  'regiao-imediata': {'id': 110005,
   'nome': 'Cacoal',
   'regiao-intermediaria': {'id': 1102,
    'nome': 'Ji-Paraná',
    'UF': {'id': 11,
     'sigla': 'RO',
     'nome': 'Rondônia',
     'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}}}},
 {'id': 1100023,
  'nome': 'Ariquemes',
  'microrregiao': {'id': 11003,
   'nome': 'Ariquemes',
   'mesorregiao': {'id': 1102,
    'nome': 'Leste Rondoniense',
    'UF': {'id': 11,
     'sigla': 'RO',
     'nome': 'Rondônia',
     'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}}},
  'regiao-imediata': {'id': 110002,
   'nome': 'Ariquemes',
   'regiao-intermediaria': {'id': 1101,
    'nome': 'Porto Velho',
    'UF': {'id': 11,
     'sigla': 'RO',
  

In [29]:
def create_county(json):
    uf, state, regiao = api.get_uf(json)
    return County(id=api.get_id(json), county=api.get_county_name(json), uf=uf, state=state, regiao=regiao)

db.create_table(engine, County)
count_records_in_county = session.query(County).count()

if count_records_in_county == 0:
    for i, county in enumerate(response.json()):
        record = create_county(county)
        session.add(record)
        session.commit()
    count_records_in_county = session.query(County).count()
    print('count records inserted in table %s: %d' % (County.__tablename__, count_records_in_county))

create table: county
count records inserted in table county: 5570


## Close connection

In [30]:
session.close()
engine.dispose()